In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, coalesce, concat, to_timestamp, when,
    last, sort_array, size, element_at, expr, collect_list, date_format
)
from pyspark.sql.window import Window
import sys
import boto3
import io
import re
# Manter o import que finalmente funcionou após as correções
from aws_encryption_sdk import EncryptionSDKClient
from aws_encryption_sdk.key_providers.kms import StrictAwsKmsMasterKeyProvider 

# ==============================================================================
# CONFIGURAÇÕES AWS E KMS
# ==============================================================================

# 🔑 ARN da chave KMS (usado na descriptografia)
KMS_KEY_ARN = "arn:aws:kms:us-east-1:462596495285:key/f6feeb08-2041-40cc-aba6-3c11fcb225f9"
KMS_REGION = 'us-east-1' # Região forçada da chave
RAW_BUCKET_NAME = "bucket-raw-upa-connect-teuscape" # Bucket onde estão os arquivos ENCRYPTED
STAGE_BUCKET_NAME = "bucket-raw-upa-connect-teuscape" # Usaremos o mesmo bucket para o staging
STAGE_PREFIX = 'temp_decrypted_staging/' # Prefixo temporário para arquivos descriptografados

# Lista de arquivos ENCRYPTED a serem processados
arquivos_atendimento_enc = [
    'ATENDIMENTOS_SUJOS_2025-10-13.csv.enc',
    'ATENDIMENTOS_SUJOS_2025-10-12.csv.enc',
    'ATENDIMENTOS_SUJOS_2025-10-11.csv.enc',
    'ATENDIMENTOS_SUJOS_2025-10-10.csv.enc',
    'ATENDIMENTOS_SUJOS_2025-10-09.csv.enc'
]

# ==============================================================================
# FUNÇÃO DE DESCRIPTOGRAFIA E STAGING
# ==============================================================================

def decrypt_and_stage_files(encrypted_files, s3_client, kms_provider):
    """
    Baixa, descriptografa e faz o upload de cada arquivo para um diretório temporário
    sem criptografia (para o Spark poder ler).
    """
    decrypted_paths = []
    
    for encrypted_key in encrypted_files:
        raw_key = encrypted_key
        staged_key = STAGE_PREFIX + encrypted_key.replace('.enc', '') # Remove o .enc
        
        print(f"\n--- Processando {encrypted_key} ---")

        # --- Passo 1: Baixar o arquivo encriptografado ---
        try:
            print(f"Baixando arquivo: s3://{RAW_BUCKET_NAME}/{raw_key}")
            s3_object = s3_client.get_object(Bucket=RAW_BUCKET_NAME, Key=raw_key)
            encrypted_data = s3_object["Body"].read() 
        except Exception as e:
            print(f"❌ ERRO ao baixar {raw_key}: {e}")
            continue

        # --- Passo 2: Descriptografar os dados ---
        try:
            print("Iniciando descriptografia...")
            encrypt_client = EncryptionSDKClient()
            decrypted_data, _ = encrypt_client.decrypt(
                source=encrypted_data,
                key_provider=kms_provider
            )
            print("✅ Descriptografia concluída.")

        except Exception as e:
            print(f"❌ ERRO ao descriptografar {raw_key}. Verifique as permissões KMS.")
            print(f"Detalhes: {e}")
            continue

        # --- Passo 3: Salvar o arquivo descriptografado no Staging (Sem .enc) ---
        try:
            print(f"Fazendo upload para staging: s3://{STAGE_BUCKET_NAME}/{staged_key}")
            s3_client.put_object(
                Bucket=STAGE_BUCKET_NAME,
                Key=staged_key,
                Body=decrypted_data
            )
            decrypted_paths.append(f"s3a://{STAGE_BUCKET_NAME}/{staged_key}")
            print(f"🎉 Staging concluído para {staged_key}")
        except Exception as e:
            print(f"❌ ERRO ao fazer upload para staging: {e}")
            continue
            
    return decrypted_paths

# ==============================================================================
# EXECUÇÃO DA DESCRIPTOGRAFIA E STAGING
# ==============================================================================

# Inicializa o cliente Boto3 para a descriptografia
session = boto3.session.Session(region_name=KMS_REGION)
s3_client = session.client("s3")

# Inicializa o provedor de chaves KMS
kms_provider = StrictAwsKmsMasterKeyProvider(
    key_ids=[KMS_KEY_ARN],
    region_names=[KMS_REGION]
)

# Executa a descriptografia e obtém a lista de caminhos para o Spark
caminhos_atendimento_dec = decrypt_and_stage_files(
    arquivos_atendimento_enc, 
    s3_client, 
    kms_provider
)

if not caminhos_atendimento_dec:
    print("\n🚨 NENHUM ARQUIVO FOI DESCRIPTOGRAFADO. ENCERRANDO ETL.")
    sys.exit(1)

print("\n--- INICIANDO PROCESSAMENTO SPARK ---")

# ==============================================================================
# CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# ==============================================================================

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).appName("TratativaAtendimentoUPA").getOrCreate()

# ==============================================================================
# DEFINIÇÃO DE ARQUIVOS E CAMINHOS
# ==============================================================================
# s3_prefixo = 's3a://bucket-raw-upa-connect-sofh/arquivos/' # <-- NÃO É MAIS USADO
s3_destino = 's3a://bucket-trusted-upa-connect-teuscape/tabela_atendimento_tratada/'

# ==============================================================================
# LEITURA E UNIFICAÇÃO DOS DADOS (AGORA DO STAGING DESCRIPTOGRAFADO)
# ==============================================================================
df_bruto = spark.read \
    .option('header', 'true') \
    .option('delimiter', ',') \
    .option('inferSchema', 'true') \
    .csv(caminhos_atendimento_dec) # <-- LENDO OS ARQUIVOS DESCRIPTOGRAFADOS

# ==============================================================================
# ETAPA 1: TRANSFORMAÇÕES INICIAIS (NOMES E COLUNAS)
# ==============================================================================
df_renomeado = df_bruto.toDF(*[c.lower() for c in df_bruto.columns])
df_renomeado = df_renomeado.withColumnRenamed("fk_pessoa", "fk_paciente")

df_com_timestamp = df_renomeado.withColumn(
    "chegou",
    coalesce(
        to_timestamp(
            concat(
                date_format(col("data"), "yyyy-MM-dd"),
                lit(" "),
                date_format(col("chegou"), "HH:mm:ss")
            ),
            "yyyy-MM-dd HH:mm:ss"
        ),
        to_timestamp(
            concat(
                date_format(col("data"), "yyyy-MM-dd"),
                lit(" "),
                date_format(col("triagem_horario"), "HH:mm:ss")
            ),
            "yyyy-MM-dd HH:mm:ss"
        )
    )
)

colunas_horario = ["triagem_horario", "sala_de_espera", "consultorio_horario", "saida"]
df_horarios_corrigidos = df_com_timestamp
for nome_coluna in colunas_horario:
    df_horarios_corrigidos = df_horarios_corrigidos.withColumn(
        nome_coluna,
        when(col(nome_coluna).isNotNull(),
            to_timestamp(
                concat(
                    date_format(col("data"), "yyyy-MM-dd"),
                    lit(" "),
                    date_format(col(nome_coluna), "HH:mm:ss")
                ),
                "yyyy-MM-dd HH:mm:ss"
            )
        )
    )

df_ordenado = df_horarios_corrigidos.orderBy("chegou")

# ==============================================================================
# ETAPA 2: TRATATIVA DE VALORES INVÁLIDOS NA COLUNA 'fk_upa'
# ==============================================================================
window_ffill = Window.orderBy("chegou").rowsBetween(Window.unboundedPreceding, 0)

df_com_upa_valida = df_ordenado.withColumn(
    "upa_valida",
    when(col("fk_upa").between(1, 34), col("fk_upa"))
)

df_com_ultimo_upa = df_com_upa_valida.withColumn(
    "ultima_upa_valida",
    last("upa_valida", ignorenulls=True).over(window_ffill)
)

df_upa_tratada = df_com_ultimo_upa.withColumn(
    "fk_upa",
    coalesce(col("upa_valida"), col("ultima_upa_valida"))
).drop("upa_valida", "ultima_upa_valida")

# ==============================================================================
# ETAPA 3: TRATATIVA DE OUTLIERS E NULOS (TEMPERATURA E OXIMETRIA)
# ==============================================================================
# --- Temperatura ---
df_temp_valida = df_upa_tratada.withColumn(
    "temp_valida",
    when((col("temperatura_paciente") >= 35) & (col("temperatura_paciente") <= 42), col("temperatura_paciente"))
)

df_com_array_temp = df_temp_valida.withColumn(
    "ultimas_3_temps",
    expr("slice(collect_list(temp_valida) OVER (ORDER BY chegou ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), -3, 3)")
)

df_com_mediana_temp = df_com_array_temp.withColumn(
    "mediana_temp",
    when(size(col("ultimas_3_temps")) > 0,
          element_at(sort_array(col("ultimas_3_temps")),
                      (size(col("ultimas_3_temps")) / 2 + 0.5).cast("int")))
)

df_temp_tratada = df_com_mediana_temp.withColumn(
    "temperatura_paciente",
    coalesce(col("temp_valida"), col("mediana_temp"))
)

# --- Oximetria ---
df_oxi_valida = df_temp_tratada.withColumn(
    "oxi_valida",
    when((col("oximetria_paciente") >= 70) & (col("oximetria_paciente") <= 100), col("oximetria_paciente"))
)

df_com_array_oxi = df_oxi_valida.withColumn(
    "ultimas_3_oxis",
    expr("slice(collect_list(oxi_valida) OVER (ORDER BY chegou ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), -3, 3)")
)

df_com_mediana_oxi = df_com_array_oxi.withColumn(
    "mediana_oxi",
    when(size(col("ultimas_3_oxis")) > 0,
          element_at(sort_array(col("ultimas_3_oxis")),
                      (size(col("ultimas_3_oxis")) / 2 + 0.5).cast("int")))
)

df_final = df_com_mediana_oxi.withColumn(
    "oximetria_paciente",
    coalesce(col("oxi_valida"), col("mediana_oxi"))
)

# ==============================================================================
# ETAPA 4: LIMPEZA FINAL E FORMATAÇÃO DE DATAS
# ==============================================================================
df_renomeado_final = df_final.withColumnRenamed("chegou", "data_hora")

# 🔹 FORMATA TODAS AS COLUNAS DE DATA/HORA NO PADRÃO YYYY-MM-DDTHH:mm:ss
df_formatado = df_renomeado_final \
    .withColumn("data_hora", date_format(col("data_hora"), "yyyy-MM-dd'T'HH:mm:ss")) \
    .withColumn("triagem_horario", date_format(col("triagem_horario"), "yyyy-MM-dd'T'HH:mm:ss")) \
    .withColumn("sala_de_espera", date_format(col("sala_de_espera"), "yyyy-MM-dd'T'HH:mm:ss")) \
    .withColumn("consultorio_horario", date_format(col("consultorio_horario"), "yyyy-MM-dd'T'HH:mm:ss")) \
    .withColumn("saida", date_format(col("saida"), "yyyy-MM-dd'T'HH:mm:ss"))

# Seleciona colunas finais
colunas_finais = [
    "data_hora",
    "id_atendimento",
    "fk_paciente",
    "triagem_horario",
    "triagem_sala",
    "sala_de_espera",
    "consultorio_horario",
    "consultorio_sala",
    "saida",
    "temperatura_paciente",
    "oximetria_paciente",
    "fk_upa"
]

tabela_unificada = df_formatado.select(colunas_finais)


print("✅ Tratamento concluído")

/usr/local/lib/python3.7/site-packages/aws_encryption_sdk/caches/__init__.py:11: CryptographyDeprecationWarning: Python 3.7 is no longer supported by the Python core team and support for it is deprecated in cryptography. The next release of cryptography will remove support for Python 3.7.
  from cryptography.hazmat.backends import default_backend



--- Processando ATENDIMENTOS_SUJOS_2025-10-13.csv.enc ---
Baixando arquivo: s3://bucket-raw-upa-connect-teuscape/ATENDIMENTOS_SUJOS_2025-10-13.csv.enc
Iniciando descriptografia...
✅ Descriptografia concluída.
Fazendo upload para staging: s3://bucket-raw-upa-connect-teuscape/temp_decrypted_staging/ATENDIMENTOS_SUJOS_2025-10-13.csv
🎉 Staging concluído para temp_decrypted_staging/ATENDIMENTOS_SUJOS_2025-10-13.csv

--- Processando ATENDIMENTOS_SUJOS_2025-10-12.csv.enc ---
Baixando arquivo: s3://bucket-raw-upa-connect-teuscape/ATENDIMENTOS_SUJOS_2025-10-12.csv.enc
Iniciando descriptografia...
✅ Descriptografia concluída.
Fazendo upload para staging: s3://bucket-raw-upa-connect-teuscape/temp_decrypted_staging/ATENDIMENTOS_SUJOS_2025-10-12.csv
🎉 Staging concluído para temp_decrypted_staging/ATENDIMENTOS_SUJOS_2025-10-12.csv

--- Processando ATENDIMENTOS_SUJOS_2025-10-11.csv.enc ---
Baixando arquivo: s3://bucket-raw-upa-connect-teuscape/ATENDIMENTOS_SUJOS_2025-10-11.csv.enc
Iniciando descript

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cb9d77b5-f6d6-485e-a527-28db64a87466;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 522ms :: artifacts dl 17ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

✅ Tratamento concluído


In [2]:
FINAL_OUTPUT_DIR = "s3a://bucket-trusted-upa-connect-teuscape/"
FINAL_FILENAME = "tabela_atendimentos_tratada.csv"
TEMP_STAGING_DIR_SPARK = f"{FINAL_OUTPUT_DIR}/_temp_staging_integrated"

# 1. Escreve o resultado no caminho temporário do Trusted (SPARK)
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR_SPARK}")

# NOTA: Coalesce(1) para garantir a geração de um único arquivo CSV.
tabela_unificada.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'UTF-8') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR_SPARK)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR_SPARK).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR_SPARK + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR_SPARK), True) 
        
        print(f"\n✅ Base integrada salva e renomeada com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# ==============================================================================
# ETAPA FINAL: LIMPEZA DOS ARQUIVOS TEMPORÁRIOS DE DESCRIPTOGRAFIA
# ==============================================================================
try:
    print(f"\nIniciando limpeza dos arquivos temporários de staging: s3://{STAGE_BUCKET_NAME}/{STAGE_PREFIX}")
    # Usa o cliente Boto3 S3 para deletar o diretório temporário
    s3_client.delete_object(Bucket=STAGE_BUCKET_NAME, Key=STAGE_PREFIX)
    # Lista e deleta todos os objetos dentro do prefixo temporário, se houver
    response = s3_client.list_objects_v2(Bucket=STAGE_BUCKET_NAME, Prefix=STAGE_PREFIX)
    if 'Contents' in response:
        delete_objects = {'Objects': [{'Key': obj['Key']} for obj in response['Contents']]}
        s3_client.delete_objects(Bucket=STAGE_BUCKET_NAME, Delete=delete_objects)
    
    print("🗑️ Arquivos temporários de descriptografia removidos com sucesso.")
except Exception as e:
    print(f"⚠️ Aviso: Falha na limpeza dos arquivos temporários de descriptografia: {e}")

# Encerra a sessão Spark
spark.stop()


Escrevendo dados temporariamente em: s3a://bucket-trusted-upa-connect-teuscape//_temp_staging_integrated


25/12/09 00:17:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/09 00:17:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/09 00:17:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/09 00:17:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/09 00:17:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/09 00:17:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/09 0


✅ Base integrada salva e renomeada com sucesso para: s3a://bucket-trusted-upa-connect-teuscape/tabela_atendimentos_tratada.csv

Iniciando limpeza dos arquivos temporários de staging: s3://bucket-raw-upa-connect-teuscape/temp_decrypted_staging/
🗑️ Arquivos temporários de descriptografia removidos com sucesso.
